## Borrower Reliability Study

The customer is the credit department of the bank. It is necessary to find out whether the marital status and the number of children of the client affect the fact of repaying the loan on time. Input data from the bank - statistics on the solvency of customers.

The results of the study will be taken into account when building a **credit scoring** model - a special system that evaluates the ability of a potential borrower to repay a loan to a bank.

The data for the study contains the following categories (columns):
- children — number of children in the family
- days_employed - work experience in days
- dob_days — client's age in years
- education - education of the client
- education_id - education ID
- family_status - marital status
- family_status_id - marital status identifier
- gender — gender of the client
- income_type - type of employment
- debt — whether he had any debt to repay loans
- total_income - income per month

In the upcoming work, we plan to familiarize ourselves with the data obtained, conduct their preprocessing and answer the following questions posed to us:
1. Is there a relationship between having children and repaying a loan on time?
2. Is there a relationship between marital status and loan repayment on time?
3. Is there a relationship between income level and loan repayment on time?
4. How do different purposes of a loan affect its repayment on time?

The content of the study is as follows:
1. [Data opening](#start)
2. [Data preprocessing](#preprocessing)
     * [Handling missing data](#null)
     * [Replacing data type](#astypecheck)
     * [Handling duplicates](#duplicates)
     * [Lemmatization](#lemcheck)
     * [Data categorization](#groupcheck)
2. [Answers to questions](#questions)

### Step 1. Data opening  <a id="start"></a>

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
df.info()
df.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Conclusion

The data sent contains information about 21,525 borrowers divided into 12 categories.
First of all, the nulls in the categories days_employed and total_income attract attention, which, preliminary, will directly affect the results of our study. Also noted are the presence of negative values in the days_employed column and different case of writing values in the education column. In any case, data preprocessing is required before the study.

### Step 2. Data preprocessing <a id="preprocessing"></a>

### Handling missing data <a id="null"></a>

In [2]:
df[(df['days_employed'].isna()) & (df['total_income'].isna())]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


In [3]:
count_days_employed = 0
for i in range(df.shape[0]):
    if (df.loc[i, 'days_employed'] < 0):
        count_days_employed += 1
count_days_employed

15906

In [4]:
df['days_employed'] = abs(df['days_employed'])
count_days_employed2 = 0
for i in range(df.shape[0]):
    if (df.loc[i, 'days_employed'] < 0):
        count_days_employed2 += 1
count_days_employed2

0

In [5]:
df_na_fill = df[['days_employed','income_type','total_income']]
df_na_fill_median = df_na_fill.groupby('income_type').median()
df_na_fill_median

,days_employed,total_income
income_type,,
безработный,366413.652744,131339.751676
в декрете,3296.759962,53829.130729
госслужащий,2689.368353,150447.935283
компаньон,1547.382223,172357.950966
пенсионер,365213.306266,118514.486412
предприниматель,520.848083,499163.144947
сотрудник,1574.202821,142594.396847
студент,578.751554,98201.625314


In [6]:
df= df.fillna(-1)
df[(df['days_employed'].isna()) & (df['total_income'].isna())].sum()

children            0.0
days_employed       0.0
dob_years           0.0
education           0.0
education_id        0.0
family_status       0.0
family_status_id    0.0
gender              0.0
income_type         0.0
debt                0.0
total_income        0.0
purpose             0.0
dtype: float64

In [7]:
for i in range(len(df)):
    if df.loc[i, 'days_employed'] == -1:
        df.loc[i, 'days_employed'] = df_na_fill_median.loc[df.loc[i, 'income_type'], 'days_employed']
    else:
        pass
for i in range(len(df)):
    if df.loc[i, 'total_income'] == -1:
        df.loc[i, 'total_income'] = df_na_fill_median.loc[df.loc[i, 'income_type'], 'total_income']
    else:
        pass

In [8]:
df.info()
df.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [9]:
df[(df['days_employed'] == -1) | (df['total_income'] == -1)].sum()

children            0.0
days_employed       0.0
dob_years           0.0
education           0.0
education_id        0.0
family_status       0.0
family_status_id    0.0
gender              0.0
income_type         0.0
debt                0.0
total_income        0.0
purpose             0.0
dtype: float64

### Conclusion

Looking at the first 20 rows of data showed NaN data. The number of rows containing NaN is 2,174. There are nulls in the days_employed and total_income columns. To fill them in, a median of values corresponding to the type of employment income_type was chosen.
Additionally, when looking at the first 20 rows of data, it was noted that most of the values in the days_employed column are negative. Counting the number of negative values showed that they indeed represent more than 3/4 of the total in the data. In this regard, the most likely reason for the occurrence of such negative values seems to be a technical recording error. To eliminate it, all values in the days_employed column were taken modulo.
Further, to continue the work on filling in the nulls, a table of values was formed to fill in by grouping by the values in the income_type column and calculating the median for the values in the days_employed and total_income columns.
For the convenience of working with nulls, they were pre-filled with "stub" values -1 (minus 1). After that, cycles were applied to replace -1 with the corresponding median value from the previously generated filling table.
Checking the results of the work showed the absence of null and their filling with correct values.

### Replacing data type <a id="astypecheck"></a>

In [10]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
df.info()
df.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


### Conclusion

During the analysis of the data, it was noticed that the values in the days_employed and total_income columns are difficult to read due to their float64 type with a large number of decimal places. To improve data readability, the type of values in these columns has been changed to int64 (integers). Considering what data the values in the columns in question refer to, discarding decimal places will have no effect on the results of the analysis.

### Handling duplicates <a id="duplicates"></a>

In [11]:
df['education'] = df['education'].str.lower()

Before processing duplicates during the previous stages of data preprocessing, it was found that in the education column the names of education types are written in different registers. All names in this column have been converted to lower case.

In [12]:
df['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [13]:
df[df['children'] == -1]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
291,-1,4417,46,среднее,1,гражданский брак,1,F,сотрудник,0,102816,профильное образование
705,-1,902,50,среднее,1,женат / замужем,0,F,госслужащий,0,137882,приобретение автомобиля
742,-1,3174,57,среднее,1,женат / замужем,0,F,сотрудник,0,64268,дополнительное образование
800,-1,349987,54,среднее,1,Не женат / не замужем,4,F,пенсионер,0,86293,дополнительное образование
941,-1,365213,57,среднее,1,женат / замужем,0,F,пенсионер,0,118514,на покупку своего автомобиля
1363,-1,1195,55,среднее,1,женат / замужем,0,F,компаньон,0,69550,профильное образование
1929,-1,1461,38,среднее,1,Не женат / не замужем,4,M,сотрудник,0,109121,покупка жилья
2073,-1,2539,42,среднее,1,в разводе,3,F,компаньон,0,162638,покупка жилья
3814,-1,3045,26,среднее,1,гражданский брак,1,F,госслужащий,0,131892,на проведение свадьбы
4201,-1,901,41,среднее,1,женат / замужем,0,F,госслужащий,0,226375,операции со своей недвижимостью


In [14]:
df[df['children'] == 20]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,880,21,среднее,1,женат / замужем,0,M,компаньон,0,145334,покупка жилья
720,20,855,44,среднее,1,женат / замужем,0,F,компаньон,0,112998,покупка недвижимости
1074,20,3310,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518,получение образования
2510,20,2714,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474,операции с коммерческой недвижимостью
2941,20,2161,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739,на покупку автомобиля
...,...,...,...,...,...,...,...,...,...,...,...,...
21008,20,1240,40,среднее,1,женат / замужем,0,F,сотрудник,1,133524,свой автомобиль
21325,20,601,37,среднее,1,женат / замужем,0,F,компаньон,0,102986,профильное образование
21390,20,1547,53,среднее,1,женат / замужем,0,M,компаньон,0,172357,покупка жилой недвижимости
21404,20,494,52,среднее,1,женат / замужем,0,M,компаньон,0,156629,операции со своей недвижимостью


In [15]:
df['children'] = df['children'].replace(-1, 1)
df['children'] = df['children'].replace(20, 2)
df['children'].unique()

array([1, 0, 3, 2, 4, 5])

An analysis of the unique values in the children column showed that there are obvious artifacts in the form of -1 and 20. A detailed analysis of the rows with these values showed a fairly large number of them without explicit binding to specific values of other columns. In the current situation, it was assumed that both values are technical input / upload errors and they were replaced by 1 and 2, respectively.

In [16]:
df = df[((df['days_employed']/247)-16) < df['dob_years']]
df.reset_index(drop=True, inplace=True)
df.info()
df.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17654 entries, 0 to 17653
Data columns (total 12 columns):
children            17654 non-null int64
days_employed       17654 non-null int64
dob_years           17654 non-null int64
education           17654 non-null object
education_id        17654 non-null int64
family_status       17654 non-null object
family_status_id    17654 non-null int64
gender              17654 non-null object
income_type         17654 non-null object
debt                17654 non-null int64
total_income        17654 non-null int64
purpose             17654 non-null object
dtypes: int64(7), object(5)
memory usage: 1.6+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
5,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
6,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
7,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
8,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи
9,2,4171,36,высшее,0,женат / замужем,0,M,компаньон,0,113943,покупка недвижимости


During the analysis and pre-processing of data, individual values of work experience in days of more than several hundred thousand were noted. Since the length of service cannot be greater than a person's age, the lines with the values of the length of service (recalculated in years, taking into account the fact that there are an average of 247 working days per year) minus the minimum age for concluding an employment contract (16 years) that exceed the age were deleted.

In [17]:
df = df[(df['dob_years'] >= 18) & (df['dob_years'] <= 70)]
df.reset_index(drop=True, inplace=True)
df.info()
df['dob_years'].unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17573 entries, 0 to 17572
Data columns (total 12 columns):
children            17573 non-null int64
days_employed       17573 non-null int64
dob_years           17573 non-null int64
education           17573 non-null object
education_id        17573 non-null int64
family_status       17573 non-null object
family_status_id    17573 non-null int64
gender              17573 non-null object
income_type         17573 non-null object
debt                17573 non-null int64
total_income        17573 non-null int64
purpose             17573 non-null object
dtypes: int64(7), object(5)
memory usage: 1.6+ MB


array([42, 36, 33, 32, 27, 43, 50, 35, 41, 40, 54, 56, 26, 48, 24, 21, 28,
       47, 34, 25, 31, 30, 20, 49, 37, 45, 61, 44, 53, 46, 23, 38, 52, 39,
       51, 62, 29, 59, 55, 64, 58, 60, 22, 63, 69, 57, 65, 19, 67, 66, 70,
       68])

Regarding the age of the client, based on the conditions of the task, compliance with the average minimum and maximum age of obtaining a loan was checked. According to a separate study of the conditions of banks, the minimum age for obtaining a loan is 18 years, the maximum is 70 years. Rows that did not meet these conditions were excluded.

In [18]:
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [19]:
df['education_id'].unique()

array([0, 1, 2, 3, 4])

In [20]:
df['family_status'] = df['family_status'].str.lower()
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'в разводе',
       'не женат / не замужем', 'вдовец / вдова'], dtype=object)

In [21]:
df['family_status_id'].unique()

array([0, 1, 3, 4, 2])

When examining the unique values of education and marital status, along with their identifiers, no artifacts were found, with the exception of one single case-distinguished marital status value.

In [22]:
df['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

In [23]:
df[df['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
8736,0,2358,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905,покупка недвижимости


In [24]:
df = df[df['gender'] != 'XNA']
df.reset_index(drop=True, inplace=True)
df.info()
df['gender'].unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17572 entries, 0 to 17571
Data columns (total 12 columns):
children            17572 non-null int64
days_employed       17572 non-null int64
dob_years           17572 non-null int64
education           17572 non-null object
education_id        17572 non-null int64
family_status       17572 non-null object
family_status_id    17572 non-null int64
gender              17572 non-null object
income_type         17572 non-null object
debt                17572 non-null int64
total_income        17572 non-null int64
purpose             17572 non-null object
dtypes: int64(7), object(5)
memory usage: 1.6+ MB


array(['F', 'M'], dtype=object)

For the gender of clients, one XNA value was found that clearly did not match any of the known categories. The line with it has been removed.

In [25]:
df['income_type'].unique()

array(['сотрудник', 'компаньон', 'госслужащий', 'предприниматель',
       'студент', 'в декрете'], dtype=object)

In [26]:
df['debt'].unique()

array([0, 1])

In [27]:
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'операции с жильем', 'образование',
       'на проведение свадьбы', 'покупка жилья для семьи',
       'покупка недвижимости', 'покупка коммерческой недвижимости',
       'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку своего автомобиля',
       'на покупку подержанного автомобиля', 'сыграть свадьбу', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'операции с коммерческой недвижимостью', 'свой автомобиль',
       'сделка с автомобилем', 'профильное образование',
       'высшее образование', 'на покупку автомобиля',
      

When analyzing the unique values of the type of employment, the presence of debt and the purpose of obtaining a loan, artifacts were also not found.

In [28]:
df.duplicated().sum()

44

In [29]:
df = df.drop_duplicates().reset_index(drop = True)
df.duplicated().sum()

0

### Conclusion

After the search and correction of artifacts in all data columns, duplicates were checked, which showed their presence in the amount of 44 pieces.
Duplicates were removed using the .drop_duplicates() method.

### Lemmatization <a id="lemcheck"></a>

In [30]:
df.info()
df.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17528 entries, 0 to 17527
Data columns (total 12 columns):
children            17528 non-null int64
days_employed       17528 non-null int64
dob_years           17528 non-null int64
education           17528 non-null object
education_id        17528 non-null int64
family_status       17528 non-null object
family_status_id    17528 non-null int64
gender              17528 non-null object
income_type         17528 non-null object
debt                17528 non-null int64
total_income        17528 non-null int64
purpose             17528 non-null object
dtypes: int64(7), object(5)
memory usage: 1.6+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
5,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
6,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
7,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
8,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи
9,2,4171,36,высшее,0,женат / замужем,0,M,компаньон,0,113943,покупка недвижимости


In [31]:
from pymystem3 import Mystem
m = Mystem()
for i in range(len(df)):
    lemma = m.lemmatize(df.loc[i, 'purpose'])
    if 'жилье' in lemma or 'недвижимость' in lemma:
        df.loc[i, 'purpose_lemm'] = 'недвижимость'
    elif 'автомобиль' in lemma:
        df.loc[i, 'purpose_lemm'] = 'автомобиль'
    elif 'образование' in lemma:
        df.loc[i, 'purpose_lemm'] = 'образование'
    elif 'свадьба' in lemma:
        df.loc[i, 'purpose_lemm'] = 'свадьба'
    else:
        pass
df.info()
df.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17528 entries, 0 to 17527
Data columns (total 13 columns):
children            17528 non-null int64
days_employed       17528 non-null int64
dob_years           17528 non-null int64
education           17528 non-null object
education_id        17528 non-null int64
family_status       17528 non-null object
family_status_id    17528 non-null int64
gender              17528 non-null object
income_type         17528 non-null object
debt                17528 non-null int64
total_income        17528 non-null int64
purpose             17528 non-null object
purpose_lemm        17528 non-null object
dtypes: int64(7), object(6)
memory usage: 1.7+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemm
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость
5,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость
6,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование
7,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба
8,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость
9,2,4171,36,высшее,0,женат / замужем,0,M,компаньон,0,113943,покупка недвижимости,недвижимость


### Conclusion

During the analysis of the unique values of the purpose column, it was noted that although the individual values are largely similar and actually belong to the same category of credit, the form of their writing does not allow them to be grouped. A visual review of the unique values made it possible to identify 4 categories of loan purposes: real estate, car, education, and marriage. At the same time, it was noted that the mention of housing should also be included in the category of real estate, which was taken into account in the analysis. To bring individual values to a common form, lemmatization was carried out with the formation of an additional column purpose_lemm, which reflects the names of loan types for the convenience of further categorization.

### Data categorization<a id="group"></a>

In [32]:
df_children = df[['children','debt']]
df_children.groupby('children').mean()

,debt
children,
0,0.081860
1,0.093544
2,0.095444
3,0.080745
4,0.100000
5,0.000000


Data categorization to answer the first question of the study was carried out with the creation of a separate table with columns children and debt with further grouping by the number of children and determining the average value of the corresponding data from the debt column. Since in the debt column the designation of the absence of debt corresponds to 0, and the presence of debt corresponds to 1, then when determining the average, the closer the result is to 1, the more likely the occurrence of debt, and vice versa for the absence of debt.

In [33]:
df_family_status_dict = df[['family_status_id','family_status']]
df_family_status_dict = df_family_status_dict.drop_duplicates().reset_index(drop=True)
df_family_status_dict.sort_values('family_status_id',ascending=True)

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
4,2,вдовец / вдова
2,3,в разводе
3,4,не женат / не замужем


In [34]:
df_family_status = df[['family_status_id','debt']]
df_family_status.groupby('family_status_id').mean().sort_values('debt',ascending=True)

,debt
family_status_id,
2,0.061905
3,0.074534
0,0.079370
1,0.100633
4,0.105070


Data categorization to answer the second question of the study was carried out by creating a separate table with the family_status_id and debt columns, further grouping by marital status identifier and determining the average value of the corresponding data from the debt column. Additionally, to decrypt the identifier, a dictionary was formed with the values of the family_status_id and family_status columns, in which duplicates were removed.

In [35]:
df_total_income = df[['total_income','debt']].copy()
total_income_min = df_total_income['total_income'].min()
total_income_min

21367

In [36]:
total_income_max = df_total_income['total_income'].max()
total_income_max

2265604

In [37]:
total_income_low = df_total_income['total_income'].quantile(.33)
total_income_low

128228.82999999999

In [38]:
total_income_high = df_total_income['total_income'].quantile(.66)
total_income_high

175101.58000000002

In [39]:
for i in range(len(df_total_income)):
    if df_total_income.loc[i, 'total_income'] <= total_income_low:
        df_total_income.loc[i, 'total_income_group'] = 'низкий'
    elif total_income_low < df_total_income.loc[i, 'total_income'] <= total_income_high:
        df_total_income.loc[i, 'total_income_group'] = 'средний'
    elif df_total_income.loc[i, 'total_income'] > total_income_high:
        df_total_income.loc[i, 'total_income_group'] = 'высокий'
    else:
        pass
df_total_income.head(20)

,total_income,debt,total_income_group
0,253875,0,высокий
1,112080,0,низкий
2,145885,0,средний
3,267628,0,высокий
4,255763,0,высокий
5,240525,0,высокий
6,135823,0,средний
7,95856,0,низкий
8,144425,0,средний
9,113943,0,низкий


In [40]:
df_total_income.groupby('total_income_group').mean().sort_values('debt',ascending=True)

,total_income,debt
total_income_group,,
высокий,267525.981376,0.076510
средний,150983.876383,0.090595
низкий,94307.125692,0.092669


The categorization of data to answer the third question of the study was carried out with the creation of a separate table with columns total_income and debt. To form the income level, quantiles were calculated in the total_income column, followed by the creation of groups: low (less than or equal to the 0.33 quintile), medium (more than the 0.33 quintile, but less than or equal to the 0.66 quintile) and high income level (more than the quintile 0.66). Further, the table was grouped according to the selected income levels and the average value from the debt column was determined for each of the groups.

In [41]:
df_purpose = df[['purpose_lemm','debt']]
df_purpose.groupby('purpose_lemm').mean().sort_values('debt',ascending=True)

,debt
purpose_lemm,
недвижимость,0.077200
свадьба,0.084261
образование,0.098201
автомобиль,0.100315


The categorization of data to answer the fourth question of the study was carried out with the creation of a separate table with columns purpose_lemm and debt with further grouping by the lemmatized purpose of the loan and determining the average value of the corresponding data from the debt column. 

### Step 3. Answers to questions<a id="questions"></a>

- Is there a relationship between having children and repaying a loan on time?

In [42]:
df_children.groupby('children').mean()

,debt
children,
0,0.081860
1,0.093544
2,0.095444
3,0.080745
4,0.100000
5,0.000000


### Conclusion

In accordance with the methodology for determining the dependence of the number of children on the presence or absence of loan debt, described above in the Data Categorization section, the indicator of the average value of the presence of debt for each category of the number of children is significantly close to 0, and in the case of having 5 children it is exactly equal to it. At the same time, the average values ​​of the presence of debt in each case (with the exception of 5 children) are close to each other and the increase in these average values ​​as the number of children grows is not linear, which indicates that there is no relationship between the presence of children and the repayment of the loan on time.

- Is there a relationship between marital status and loan repayment on time?

In [43]:
df_family_status.groupby('family_status_id').mean().sort_values('debt',ascending=True)

,debt
family_status_id,
2,0.061905
3,0.074534
0,0.079370
1,0.100633
4,0.105070


### Conclusion

According to the results of work in the Data Categorization section, it can be noted that when the results of calculating the average values of loan debt in ascending order, there is an increase in the presence of debt in the following order: widower / widow, divorced, married / married, civil marriage, single / single, which suggests that the presence of a spouse or spouse affects the presence of debt, reducing the likelihood of its formation. Most likely this is due to the fact that if a client has difficulties in repaying a loan, in most cases he can be supported by a spouse without forming a debt. The presence of low debt among widows/widowers and divorced clients is probably due to their great life experience and readiness to faithfully fulfill loan obligations.

- Is there a relationship between income level and loan repayment on time?

In [44]:
df_total_income.groupby('total_income_group').mean().sort_values('debt',ascending=True)

,total_income,debt
total_income_group,,
высокий,267525.981376,0.076510
средний,150983.876383,0.090595
низкий,94307.125692,0.092669


### Conclusion

According to the results of the study, low and medium income levels are close in terms of loan repayment on time, but we can say that the presence of high income reduces the likelihood of debt, which is obvious, since if the client has more income, there are much more opportunities for timely payment of the loan .

- How do different purposes of a loan affect its repayment on time?

In [45]:
df_purpose.pivot_table(index = 'purpose_lemm', values = 'debt').sort_values('debt',ascending=True)

,debt
purpose_lemm,
недвижимость,0.077200
свадьба,0.084261
образование,0.098201
автомобиль,0.100315


### Conclusion

Visually, according to the results of the study, two groups of similar goals for obtaining a loan are distinguished: the first is real estate and a wedding, the second is education and a car. At the same time, it is noteworthy that loans for a wedding, and especially real estate, are the most reliable for banks, presumably due to the fact that a real estate loan is taken for a significant amount by clients prepared for a serious procedure for registering and buying real estate, and a wedding loan for a relatively small amount and term, which enables the client to pay it on time without difficulty. In turn, the greater likelihood of debt on loans for education and a car can be influenced by the age and final thoughtlessness of the actions of clients - usually these are the first loans in life, issued for things that are usually considered "mandatory" in a person's life and which "everyone should have".

### Step 4. General conclusion

Based on the results of the entire analysis and research, we can say that when assessing the ability of a potential borrower to repay a loan to a bank, one should first of all take into account the marital status, income level and purpose of the loan from the client. At the same time, the presence and number of children of the client does not actually affect the results of such an assessment. It is noted that clients with a high level of income, who have or had a spouse and take a loan to purchase real estate, have the greatest ability to repay a loan. In turn, the least suitable type for issuing a loan is a client with a low-to-middle income, single and buying a car.